# Объединение данных

In [1]:
import pandas as pd
import pandasql as ps
import time 

Используем два набора данных

In [2]:
Data1 = pd.read_csv("user_device.csv")
Data2 = pd.read_csv("user_usage.csv")

In [3]:
Data1.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [4]:
Data2.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [5]:
print(f"Data1: {Data1.shape}")
print(f"Data2: {Data2.shape}")

Data1: (272, 6)
Data2: (240, 4)


Для "склеивания" используем "Inner Merge"

In [6]:
start=time.time()
result = pd.merge(Data2, Data1[['use_id', 'platform', 'device']], 
                  on='use_id', how='inner', indicator=True)
end = time.time()
print('Time: ', end-start)

Time:  0.014614105224609375


In [18]:
result.shape

(159, 7)

In [8]:
sqlqur = f'''SELECT * FROM Data1 join Data2 on Data1.use_id=Data2.use_id'''

In [9]:
start=time.time()
avg = ps.sqldf(sqlqur, locals())
end=time.time()
print('Time: ', end-start)

Time:  0.03294801712036133


In [19]:
avg.shape

(159, 10)

In [12]:
start=time.time()
df=Data1.groupby(by='platform')
end=time.time()
print('Time: ', end-start)

Time:  0.0003590583801269531


In [15]:
k=df.groups.keys()

for key in k:
    print('Key:', key)
    print(df.get_group(key).count(),'\n')

Key: android
use_id              184
user_id             184
platform            184
platform_version    184
device              184
use_type_id         184
dtype: int64 

Key: ios
use_id              88
user_id             88
platform            88
platform_version    88
device              88
use_type_id         88
dtype: int64 



In [13]:
sqlgroup = 'SELECT platform, count(use_id) from Data1 group by platform'
start=time.time()
gr=ps.sqldf(sqlgroup, locals())
end=time.time()
print('Time: ', end-start,'\n')
print(gr)

Time:  0.011040687561035156 

  platform  count(use_id)
0  android            184
1      ios             88
